In [60]:
import numpy as np
from scipy import linalg as la

### Task 25
SVD and QR factorization

$A = QR$ such that $Qy = b$ and $Rx = y$

$Q$ is orthonormal and $R$  is upper triangular

In [61]:
def is_ortogonal(M):
    I1 = np.transpose(M) @ M
    I1 = I1.round(0)
    I2 = np.eye(len(I1), len(I1[0]))

    if np.array_equal(I1, I2):
        return True
    else:
        return False

In [62]:
m = 8
n = 4

QR decomposition

In [63]:
A = np.random.rand(m,n)
Q_np, R_np = np.linalg.qr(A, 'reduced')
Q_sp, R_sp = la.qr(A, mode='economic')

print("Q_np is othogonal: ", is_ortogonal(Q_np))
print("Q_sp is othogonal: ", is_ortogonal(Q_sp))

Q_np is othogonal:  True
Q_sp is othogonal:  True


Reduced mode (numpy) returns Q and R with dimensions $(M, K)$ $(K, N)$ whereas $K = min(N, M)$
and economic mode (scipy) is ...

SVD (Single Value Decomposition)

In [64]:
U_np, S_np, VH_np = np.linalg.svd(A)
U_sp, S_sp, VH_sp = la.svd(A)